In [1]:
from data import get_datasets
from model import DysClassifier
import torch.optim as optim
from torch.utils.data import default_collate
from torch import nn
from tqdm import tqdm

In [5]:
train, test, val = get_datasets('data/dys/', 'data/not_dys/', batch_size=32)#, val=True, test_val_part=[.2])

In [6]:
model = DysClassifier()
model.eval()

DysClassifier(
  (_DysClassifier__backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
   

In [7]:
n_epochs = 200
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()

for epoch in range(n_epochs):
    pbar = tqdm(train)
    for X_batch, y_batch in pbar:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        pbar.desc = f'[{epoch+1}/{n_epochs}] loss: {loss}'
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
# create one test tensor from the testset
X_test, y_test = default_collate(test.dataset)
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))

  0%|          | 0/80 [00:00<?, ?it/s]

[1/200] loss: 0.5550574064254761:   2%|▎         | 2/80 [00:25<16:53, 12.99s/it]

In [ ]:
y_pred.size()

torch.Size([1, 1])

In [6]:
y_batch.size()

torch.Size([1, 2533])